In [3]:
#장애인 콜 시스템 분석
import requests
import pprint
import json
import pandas as pd
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#API KEY
my_key = '46454c487563686139374b4a4a486f'

start = '20220101'
end = '20220501'

dt_index = pd.date_range(start = start, end = end)
dt_list = dt_index.strftime("%Y%m%d").tolist()


for i in dt_list:
    endpoint=f"http://openapi.seoul.go.kr:8088/46454c487563686139374b4a4a486f/xml/disabledCalltaxi/1/5/" + i
    resp=requests.get(endpoint)
    xml_obj = bs4.BeautifulSoup(resp.text,'lxml-xml')
    rows = xml_obj.findAll('item')
    # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
    row_list = [] # 행값
    name_list = [] # 열이름값
    value_list = [] #데이터값
    # xml 안의 데이터 수집
    for k in range(0, len(rows)):
          columns = rows[k].find_all()
          for j in range(0,len(columns)):
            if k ==0:
              name_list.append(columns[j].name)
            value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]

Calltaxi = pd.DataFrame(row_list, columns=name_list)



ValueError: ignored

In [ ]:
# 샘플 url 변형해서 사용
endpoint=f"http://openapi.seoul.go.kr:8088/46454c487563686139374b4a4a486f/xml/disabledCalltaxi/1/5/20190521"

#api 호출
resp=requests.get(endpoint)
print(resp.status_code)
print(resp.text)


xml_obj = bs4.BeautifulSoup(resp.text,'lxml-xml')
rows = xml_obj.findAll('item')
print(rows)


In [ ]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [ ]:
Calltaxi = pd.DataFrame(row_list, columns=name_list)
print(Calltaxi)

       no cartype             receipttime                 settime  \
0    5790    중형승합  2019-05-21 오전 12:05:00  2019-05-21 오전 12:05:38   
1    5799    중형승합  2019-05-21 오전 12:14:00  2019-05-21 오전 12:16:40   
2    7655    중형승합  2019-05-21 오전 12:28:00  2019-05-21 오전 12:55:05   
3    5790    중형승합  2019-05-21 오전 12:29:50  2019-05-21 오전 12:31:20   
4    9540    중형승합  2019-05-21 오전 12:49:00   2019-05-21 오전 1:33:59   
..    ...     ...                     ...                     ...   
244  5697    중형승합   2019-05-21 오전 7:00:00   2019-05-21 오전 7:10:24   
245  6015    중형승용   2019-05-21 오전 7:00:00   2019-05-21 오전 7:01:08   
246  1101    중형승합   2019-05-21 오전 7:00:00   2019-05-21 오전 7:07:14   
247  1101    중형승합   2019-05-21 오전 7:00:00   2019-05-21 오전 8:12:32   
248   824    중형승합   2019-05-21 오전 7:00:00   2019-05-21 오전 8:13:03   

                   ridetime startpos1    startpos2 endpos1 endpos2  
0    2019-05-21 오전 12:15:19      동대문구        전농제2동     노원구  중계2.3동  
1    2019-05-21 오전 12:36:07      

In [ ]:
Calltaxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   no           249 non-null    object
 1   cartype      249 non-null    object
 2   receipttime  249 non-null    object
 3   settime      249 non-null    object
 4   ridetime     249 non-null    object
 5   startpos1    249 non-null    object
 6   startpos2    249 non-null    object
 7   endpos1      249 non-null    object
 8   endpos2      249 non-null    object
dtypes: object(9)
memory usage: 17.6+ KB


In [ ]:
pal = pd.read_excel("location.xlsx")

pal = pal[pal.시도 =='서울특별시']
pal = pal[['코드','시군구','읍면동','위도','경도']]
pal = pal.rename(columns={'읍면동':'동'})
pal['동']=pal['동'].str.strip("제")

pal = pal.replace('창신제1동','창신1동')
pal = pal.replace('창신제2동','창신2동')
pal = pal.replace('창신제3동','창신3동')
pal = pal.replace('숭인제1동','숭인1동')
pal = pal.replace('숭인제2동','숭인2동')

pal = pal.apply(lambda x: x.str.strip("제"), axis = 1)
pal['동'].replace('제', '', inplace=True)

data = pd.merge(pal, Calltaxi, on="동")
data

In [ ]:
import folium
from folium import plugins

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

geo_data = 'TL_SCCO_SIG.json'

m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12, 
    tiles='cartodbpositron'
) 

folium.GeoJson(
    seoul_geo,
    name='지역동'
).add_to(m)

marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(data['위도'], data['경도']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)

m

map_data = folium.Map([35.765788, 128.619197], zoom_start=10)
lines = route_31_pos.values[:].tolist()

folium.PolyLine( 
    locations=lines,
    fill=True,
    color='orange').add_to(map_data)

map_data